In [ ]:
!python --version

# Install detectron2

In [ ]:
!pip install pyyaml==5.1
# test
!pip uninstall torch -y
!pip uninstall torchvision -y
!pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 -f https://download.pytorch.org/whl/torch_stable.html
#testend

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
os.getcwd()
!mkdir workDir
!mount --bind /content/drive/MyDrive/Uni/Masterarbeit/ML/ /content/workDir

In [ ]:
from detectron2.data.datasets import register_coco_instances 
DatasetCatalog.clear()
register_coco_instances("my_dataset_train", {}, "/content/workDir/train_img_small_complete/annotations/instances_default.json", "/content/workDir/train_img_small_complete/images")
register_coco_instances("my_dataset_val", {}, "/content/workDir/cvat_annotations_tiles/val_small_img/annotations/instances_default.json", "/content/workDir/ccvat_annotations_tiles/val_small_img/images")
register_coco_instances("my_dataset_test", {}, "/content/workDir/cvat_annotations_tiles/test_small_img/annotations/instances_default.json", "/content/workDir/cvat_annotations_tiles/test_small_img/images")



In [ ]:
# did not work correctly. the problem seems to lie with cv_imshow()

from detectron2.data import get_detection_dataset_dicts
dataset_dicts = get_detection_dataset_dicts("my_dataset_train")
for d in random.sample(dataset_dicts, 5):
    img = cv2.imread(d['file_name'])
    print(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], scale=0.7)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:

building_metadata = MetadataCatalog.get("my_dataset_train")

for i, d in enumerate(random.sample(dataset_dicts, 5)):
    # read the image with cv2

    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=building_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
# the actual training
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train", )
cfg.DATASETS.TEST = () 
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2 
cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR OG: 0.00025
cfg.SOLVER.MAX_ITER = 5000    
cfg.SOLVER.STEPS = [0.000001] 
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # "greenroof" is the only class
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

## Inference & evaluation using the trained model



In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:


cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/workDir/model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
#Evaluation is not working atm

#from detectron2.utils.visualizer import ColorMode
#dataset_dicts = get_detection_dataset_dicts("my_dataset_val" )
#for d in random.sample(dataset_dicts, 5):
#    print(d["file_name"])    
#    im = cv2.imread(d["file_name"])
#    print(d["file_name"], "test")
#    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
#    v = Visualizer(im[:, :, ::-1],
#                   scale=1, # og:0.5
                   #instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
#    )
#    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#    cv2_imshow(out.get_image()[:, :, ::-1])

Test for Images for visual confirmation of the model working:

In [ ]:
import os
import copy
import json
from detectron2.utils.visualizer import ColorMode

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/workDir/model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

for i in range(44):
  folder = str(i)
  print(f"folder: {folder}")
  if not os.path.exists(f"/content/workDir/bilder_test/1/greenroof_output_{folder}.json"):


    #test_sample = os.scandir("/content/workDir/Test_Images/images/")

    test_sample = os.listdir(f"/content/workDir/bilder_test/{folder}/")
    errorCount = 0

    prediction_outputs = []
    for ts in test_sample:
      #print(ts)
      try:
        im = cv2.imread(os.path.join(f"/content/workDir/bilder_test/{folder}/", ts))

        outputs= predictor(im)
        # print(outputs['instances'].pred_masks)
        if (len(outputs["instances"].pred_masks) > 0):
          print(f'{len(outputs["instances"].pred_masks)} x {len(outputs["instances"].pred_masks[0])} x {len(outputs["instances"].pred_masks[0][0])}')
          #print(sum(outputs["instances"].pred_boxes.area()))
          s = 0
          # laufe durch alle gefundenen sachen
          for index_of_prediction in range(len(outputs["instances"].pred_masks)):
            # wenn es gefundene klassen gibt nimm nur tensors mit mehr als x confidence
            # möglicher weise nochmal ergänzen nach klasse filtern
            prediction_mask = outputs["instances"].pred_masks[index_of_prediction]
            # print(sum(map(lambda b : b.area(),deepCopy["instances"].pred_boxes[c])))
            if outputs["instances"].scores[index_of_prediction] > cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST and outputs["instances"].pred_classes[index_of_prediction] == 0:
              # print(deepCopy["instances"].scores[index_of_prediction])
              for rows_of_prediction in prediction_mask:
                for prediction_value_at_pixel in rows_of_prediction:
                  if prediction_value_at_pixel:
                    s = s+1
          # print size of prediction mask (in pixels i guess)
         
          print(f'{ts}: {s} px')
          prediction_outputs.append({"filename": ts, "greenroof_pixelcount": s})
      except Exception as e:
        print(f'{type(e).__name__}: {e}')
        errorCount += 1
        # print(f'error with {ts}. passing on')
        continue
      #v = Visualizer(im[:, :, ::-1],
      #                 scale=1, # og:0.5
      #                 instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
      #   )
      #out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
      #cv2_imshow(out.get_image()[:, :, ::-1])

    #print(outputs)
    print(f'erros: {errorCount}')
    with open(f"/content/workDir/bilder_test/1/greenroof_output_{folder}.json", "w") as output_file:
      json.dump(prediction_outputs, output_file)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

In [ ]:
cp -r /content/output /content/workDir/output

